##### 0) 라이브러리 불러오기

In [1]:
# (1) 기본 라이브러리
import pandas as pd
import konlpy
import nltk
from konlpy.tag import Okt

# (1) 딥러닝 라이브러리
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bang\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
"""
1. CSV 파일 읽기
2. 문장수 계산
    1) for문 돌려 - 문장수 계산
        (1) content열 하나 불러오기
        (2) sentence열 하나 만들어서 문장 수 저장
3. 문장 수를 입력으로, 해당 글의 글쓰기 평가 점수 예측 모델 구현
    1) for문 돌려 - 평가 점수 예측
        (1) sentence열을 input, score열을 output해서 불러오기
        (2) 초기 세팅
            1] 모델 초기화
            2] Opitmizer 설정
            3] 에포크 설정
        (3) 에포크 돌음
            1] 예측값 계산
            2] loss 계산
            3] 역전파
            4] Optimizer 다음 스텝
4. 문장 수 + Phrase 수를 입력으로, 해당 글의 글쓰기 평가점수를 예측 모델 구현
"""

'\n1. CSV 파일 읽기\n2. 문장수 계산\n    1) for문 돌려 - 문장수 계산\n        (1) content열 하나 불러오기\n        (2) sentence열 하나 만들어서 문장 수 저장\n3. 문장 수를 입력으로, 해당 글의 글쓰기 평가 점수 예측 모델 구현\n    1) for문 돌려 - 평가 점수 예측\n        (1) sentence열을 input, score열을 output해서 불러오기\n        (2) 초기 세팅\n            1] 모델 초기화\n            2] Opitmizer 설정\n            3] 에포크 설정\n        (3) 에포크 돌음\n            1] 예측값 계산\n            2] loss 계산\n            3] 역전파\n            4] Optimizer 다음 스텝\n4. 문장 수 + Phrase 수를 입력으로, 해당 글의 글쓰기 평가점수를 예측 모델 구현\n'

##### 1) 함수 정의

In [3]:
def one_hot_encoding(word, word_to_index):
    one_hot_vector = [0] * (len(word_to_index))
    # print(len(one_hot_vector))
    # print(word)
    index = word_to_index[word]
    # print(index)
    one_hot_vector[index] = 1
    return one_hot_vector

##### 2) 데이터 불러오기

In [4]:
df = pd.read_csv("./AWE_1000_score.csv")

In [5]:
df

,Unnamed: 0,id,nationality,sex,title,content,score
0,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80
1,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70
2,3,A100000_v03.txt_1.txt,중국,남자,여행 계획,저는 여행을 제주도에서 갈 싶습니다. 저는 친구 같이 갈 겁니다. 6월 15일 갈 ...,50
3,4,A100001_v01.txt_1.txt,중국,남자,주말 이야기,주말에 도서관에서 공부했어요. 친구하고 노래방에서 노래했어요. 친구하고 식당에서 비...,60
4,5,A100001_v02.txt_1.txt,중국,남자,주말 이야기,어제 저는 학교에 갔습니다. 학교에서 친구하고를 다닙니다. 학교에 학생 식당 있습니다.,20
...,...,...,...,...,...,...,...
995,1007,A200079_v01.txt_1.txt,일본,여자,사진기 빌리기,하지만 빌리 씨는 사진기를 없었습니다. 그래서 빌리 씨는 아노코 씨에게 이야기하고 ...,90
996,1008,A200079_v01.txt_2.txt,일본,여자,우리 나라의 계절과 날씨,내 나라는 일본입니다. 일본의 계절은 뚜렷합니다. 봄은 다뜻합니다. 꽃을 많이 피나...,80
997,1009,A200079_v02.txt_1.txt,스웨덴,남자,편지,잘 지냈어요? 저도 잘 지냈어요. 한국 생활은 아주 재미있고 편해요. 한국어 공부를...,80
998,1010,A200079_v02.txt_2.txt,스웨덴,남자,기억에 남는 여행,지난 주말에 친구들하고 이탈리아에서 축구 경기를 보러 갔습니다. 우리는 영국에서 비...,90


In [6]:
df["content"]

0      제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...
1      어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...
2      저는 여행을 제주도에서 갈 싶습니다. 저는 친구 같이 갈 겁니다. 6월 15일 갈 ...
3      주말에 도서관에서 공부했어요. 친구하고 노래방에서 노래했어요. 친구하고 식당에서 비...
4       어제 저는 학교에 갔습니다. 학교에서 친구하고를 다닙니다. 학교에 학생 식당 있습니다.
                             ...                        
995    하지만 빌리 씨는 사진기를 없었습니다. 그래서 빌리 씨는 아노코 씨에게 이야기하고 ...
996    내 나라는 일본입니다. 일본의 계절은 뚜렷합니다. 봄은 다뜻합니다. 꽃을 많이 피나...
997    잘 지냈어요? 저도 잘 지냈어요. 한국 생활은 아주 재미있고 편해요. 한국어 공부를...
998    지난 주말에 친구들하고 이탈리아에서 축구 경기를 보러 갔습니다. 우리는 영국에서 비...
999    하지만 빌리 씨는 사진기가 없었어요. 그 이야기를 나오코씨에게 이야기했어요. 그래서...
Name: content, Length: 1000, dtype: object

In [7]:
df["content"][0]

'제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 많이 있습니다. 사람들이 공원에 가서 꽃을 구경해요. 사진을 찍거나 놀라요. 여름에는 아주 더워요. 그래서 사람들이 많이 수영을 하러 수영장에 가요. 여름은 아이스크림 많이 있어요. 사람이 아주 좋아해요. 가을에는 날씨가 쌀쌀해요. 바람이 많이 불러요. 단풍을 필습니다. 그래서 산을 가서 단풍을 구경해요. 겨울에는 아주 춥니다. 눈이 많이 내려요. 그럼 사림들이 눈사람을 만나고 스키장에 가요.'

In [8]:
df["content"][0].split(".")

['제 나라의 계절은 아주 좋아요',
 ' 네 계절이 있습니다',
 ' 봄에는 날씨가 좋아요',
 ' 꽃이 많이 있습니다',
 ' 사람들이 공원에 가서 꽃을 구경해요',
 ' 사진을 찍거나 놀라요',
 ' 여름에는 아주 더워요',
 ' 그래서 사람들이 많이 수영을 하러 수영장에 가요',
 ' 여름은 아이스크림 많이 있어요',
 ' 사람이 아주 좋아해요',
 ' 가을에는 날씨가 쌀쌀해요',
 ' 바람이 많이 불러요',
 ' 단풍을 필습니다',
 ' 그래서 산을 가서 단풍을 구경해요',
 ' 겨울에는 아주 춥니다',
 ' 눈이 많이 내려요',
 ' 그럼 사림들이 눈사람을 만나고 스키장에 가요',
 '']

In [9]:
len(df["content"][0].split("."))

18

##### 3) 한국어 말뭉치 선언

In [10]:
okt = Okt()

In [11]:
def build_bag_of_words(document):
  # (1) 불용어 제거
  document = document.replace('.', '')
  # (2) 형태소 추출
  tokenized_document = okt.morphs(document)

  # (3) 변수 선언
  word_to_index = {}
  bow = []

  # (4) for문 순회
  for word in tokenized_document:
    # 1] word_to_index Dictionary에 존재하지 않는 단어인 경우 : word_to_index Dictionary 생성 + BoW 기본값 1 설정
    if word not in word_to_index.keys():
      word_to_index[word] = len(word_to_index)  
      bow.insert(len(word_to_index) - 1, 1)
    # 2] word_to_index Dictionary에 존재하는 단어인 경우 : word_to_index Dictionary 내의 index 증가 + BoW index 증가
    else:
      index = word_to_index.get(word)
      bow[index] = bow[index] + 1
  
  return word_to_index, bow

##### 4) Sentence 길이 추출

In [12]:
"""
1) for문 돌려 - 문장수 계산
    (1) content열 하나 불러오기
    (2) sentence열 하나 만들어서 문장 수 저장
"""

'\n1) for문 돌려 - 문장수 계산\n    (1) content열 하나 불러오기\n    (2) sentence열 하나 만들어서 문장 수 저장\n'

In [13]:
df["sentence_len"] = df["content"].apply(lambda x: len(x.split("."))) # !!!
# sentence_len = []
# for i in range(len(df)):
#     print(len(df["content"][i].split(".")))
#     sentence_len.append(len(df["content"][i].split(".")))

In [14]:
df["sentence_len"]

0      18
1      10
2       7
3       9
4       4
       ..
995     7
996    15
997    11
998    14
999     9
Name: sentence_len, Length: 1000, dtype: int64

In [15]:
len(df["sentence_len"])

1000

# 1. 입력 : 문장수 -> 출력 : 평가 점수

In [22]:
"""
1) for문 돌려 - 평가 점수 예측
    (1) sentence열을 input, score열을 output해서 불러오기
    (2) 초기 세팅
        1] 모델 초기화
        2] Opitmizer 설정
        3] 에포크 설정
    (2) 에포크 돌음
        1] 예측값 계산
        2] loss 계산
        3] 역전파
        4] Optimizer 다음 스텝
"""

'\n1) for문 돌려 - 평가 점수 예측\n    (1) sentence열을 input, score열을 output해서 불러오기\n    (2) 초기 세팅\n        1] 모델 초기화\n        2] Opitmizer 설정\n        3] 에포크 설정\n    (2) 에포크 돌음\n        1] 예측값 계산\n        2] loss 계산\n        3] 역전파\n        4] Optimizer 다음 스텝\n'

In [23]:
df["score"].values

array([ 80,  70,  50,  60,  20,  70,  50,  80,  90,  80,  80,  20,  70,
        90,  80,  90,  70,  90,  80,  60,  70, 100,  70,  90,  80,  70,
        90,  90,  60,  90,  80,  60,  80,  80,  60,  80,  50,  70,  70,
        70,  70,  90,  70,  70,  80,  90,  90,  80,  90,  80, 100,  90,
        80,  90,  70,  70,  70,  90,  80,  70,  80,  80,  80, 100,  70,
        60,  80,  60,  60,  90,  80,  70,  90,  40,  60,  90,  50,  80,
        40,  50,  80,  80,  60,  90,  70,  60,  90,  90,  20,  90,  80,
        80,  90,  80,  90,  50,  80,  90,  40,  50,  80,  60,  90,  90,
        80,  70,  80,  30,  40,  80,  90,  50,  90,  90,  50,  90,  90,
         0,  80, 100,  60,  60,  90,  80,  70,  90,  70,  90,  80,  80,
        70,  90,  70,  40,  90,  80,  40,  90,  90,  90,  80,  90,  90,
       100,  80,  10,  90,  80,  90,  90,  60,  80,  90,  90,  40,  80,
        80,  60,  80,  50,  60,  60,  60,  80,  90,  80,  70,  80,  90,
        50,  90,  70,   0,  80,  60,   0,  80,  50,  60,  70,  7

In [24]:
df["sentence_len"].values

array([18, 10,  7,  9,  4, 11,  8, 10,  9, 13, 14,  4, 10, 10, 10, 13,  8,
       12, 11,  9, 12, 10,  7, 13, 13, 12, 10, 10,  9, 13, 11,  9, 13, 11,
        9, 11, 10,  9,  9, 10, 10, 11,  8,  9, 13, 10, 10,  9, 13, 10,  8,
       10, 10, 11, 11,  9, 10,  9, 12,  9,  9,  9,  9, 10, 11,  9,  9, 15,
       10, 11, 13, 15,  9,  7,  9, 10,  9,  9, 10,  8, 13, 12, 12, 10, 12,
        7, 12, 10, 11, 11,  8,  9, 10,  9, 10, 11,  9,  9,  7,  7,  8, 10,
       11, 12, 11, 10,  9, 10, 18, 10, 10,  5,  9, 12, 13,  9, 10, 11, 11,
       18, 15,  8, 11, 10, 11,  9,  9, 10,  9,  8, 10, 10, 12,  9, 13, 14,
        7, 13, 11, 13, 17,  9,  9, 14,  7,  9, 11,  7,  9, 10,  8,  9, 13,
        8,  5,  9, 10, 11, 11,  7, 10,  7, 13, 12,  9, 11,  9, 17, 10,  9,
       11, 12,  3, 11,  9,  7,  9,  8, 12, 12, 10, 15,  9, 12, 11,  7, 11,
       10,  8,  8,  9,  8,  6, 11, 12, 11,  9, 11, 10,  9,  9, 10, 11,  9,
        4, 13, 13, 10, 10, 13, 10, 11, 10, 18,  9, 11, 12, 10, 12, 16, 11,
       11, 13, 12,  9, 13

In [25]:
torch.tensor(df["score"].values).reshape(-1, 1)

tensor([[ 80],
        [ 70],
        [ 50],
        [ 60],
        [ 20],
        [ 70],
        [ 50],
        [ 80],
        [ 90],
        [ 80],
        [ 80],
        [ 20],
        [ 70],
        [ 90],
        [ 80],
        [ 90],
        [ 70],
        [ 90],
        [ 80],
        [ 60],
        [ 70],
        [100],
        [ 70],
        [ 90],
        [ 80],
        [ 70],
        [ 90],
        [ 90],
        [ 60],
        [ 90],
        [ 80],
        [ 60],
        [ 80],
        [ 80],
        [ 60],
        [ 80],
        [ 50],
        [ 70],
        [ 70],
        [ 70],
        [ 70],
        [ 90],
        [ 70],
        [ 70],
        [ 80],
        [ 90],
        [ 90],
        [ 80],
        [ 90],
        [ 80],
        [100],
        [ 90],
        [ 80],
        [ 90],
        [ 70],
        [ 70],
        [ 70],
        [ 90],
        [ 80],
        [ 70],
        [ 80],
        [ 80],
        [ 80],
        [100],
        [ 70],
        [ 60],
        [ 

In [26]:
torch.tensor(df["sentence_len"].values).reshape(-1, 1)

tensor([[18],
        [10],
        [ 7],
        [ 9],
        [ 4],
        [11],
        [ 8],
        [10],
        [ 9],
        [13],
        [14],
        [ 4],
        [10],
        [10],
        [10],
        [13],
        [ 8],
        [12],
        [11],
        [ 9],
        [12],
        [10],
        [ 7],
        [13],
        [13],
        [12],
        [10],
        [10],
        [ 9],
        [13],
        [11],
        [ 9],
        [13],
        [11],
        [ 9],
        [11],
        [10],
        [ 9],
        [ 9],
        [10],
        [10],
        [11],
        [ 8],
        [ 9],
        [13],
        [10],
        [10],
        [ 9],
        [13],
        [10],
        [ 8],
        [10],
        [10],
        [11],
        [11],
        [ 9],
        [10],
        [ 9],
        [12],
        [ 9],
        [ 9],
        [ 9],
        [ 9],
        [10],
        [11],
        [ 9],
        [ 9],
        [15],
        [10],
        [11],
        [13],
      

##### 5) train 데이터 불러오기

In [28]:
X_train = torch.tensor(df["sentence"].values).reshape(-1, 1).to(torch.float32)
y_train = torch.tensor(df["score"].values).reshape(-1, 1).to(torch.float32)

##### 6) 모델 정의 (SingleVariableLinearRegression)

In [27]:
class SingleVariableLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
    def forward(self, x):
        return self.linear(x)
    
model = SingleVariableLinearRegressionModel()

##### 7) Optimizer 정의

In [29]:
optimizer = optim.SGD(model.parameters(), lr=0.00005, momentum=0.9)

##### 8) 모델 학습

In [30]:
# (1) Epoch 순회
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # 1] 모델 예측
    y_pred = model(X_train)
    # 2] Cost 계산
    cost = F.mse_loss(y_pred, y_train)
    
    # 3] 모델 역전파
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 4] 결과 출력
    print('Epoch {:4d}/{} Cost:{:.6f}'.format(epoch, nb_epochs, cost.item())) #  hypothesis: {} , pred.squeeze().detach()

Epoch    0/20 Cost:4613.681641
Epoch    1/20 Cost:4516.086426
Epoch    2/20 Cost:4335.153809
Epoch    3/20 Cost:4087.198975
Epoch    4/20 Cost:3789.362305
Epoch    5/20 Cost:3458.566162
Epoch    6/20 Cost:3110.700439
Epoch    7/20 Cost:2760.011475
Epoch    8/20 Cost:2418.692139
Epoch    9/20 Cost:2096.651611
Epoch   10/20 Cost:1801.438599
Epoch   11/20 Cost:1538.298218
Epoch   12/20 Cost:1310.325439
Epoch   13/20 Cost:1118.694702
Epoch   14/20 Cost:962.939514
Epoch   15/20 Cost:841.259277
Epoch   16/20 Cost:750.830750
Epoch   17/20 Cost:688.111633
Epoch   18/20 Cost:649.122498
Epoch   19/20 Cost:629.696716
Epoch   20/20 Cost:625.694946


# 2. 입력 : 문장 수 + Phrase 수 -> 출력 : 평가 점수

##### 5) Phrase 개수 추출

(1) 구 정의 (명사구(NP) / 동사구(VP) / 형용사구(AP))

In [31]:
grammar = """ # !!!
NP: {<N.*>*<Suffix>?}   # Noun phrase
VP: {<V.*>*}            # Verb phrase
AP: {<A.*>*}            # Adjective phrase
"""

(2) Phrase 개수 추출 (Chunks 길이 추출)

In [32]:
phrase_len = []
for i in range(len(df)):
    words = konlpy.tag.Twitter().pos(df["content"][i]) # !!!
    parser = nltk.RegexpParser(grammar) # !!!
    chunks = parser.parse(words) # !!!
    phrase_len.append(len(chunks))
#     print(f"data["content"][i] : {data["content"][i]}")
#     print(f"chunks : {chunks}")
#     print(f"len(chunks) : {len(chunks)}")

C:\Users\Bang\Anaconda\envs\BangEnv\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [33]:
phrase_len

[105,
 40,
 40,
 51,
 16,
 64,
 51,
 45,
 74,
 68,
 86,
 15,
 57,
 56,
 67,
 56,
 42,
 104,
 71,
 38,
 74,
 76,
 38,
 99,
 108,
 59,
 64,
 59,
 48,
 86,
 73,
 38,
 77,
 62,
 41,
 81,
 67,
 63,
 79,
 49,
 56,
 75,
 41,
 65,
 95,
 50,
 69,
 48,
 81,
 90,
 72,
 67,
 57,
 60,
 69,
 59,
 76,
 49,
 77,
 39,
 54,
 68,
 59,
 74,
 48,
 46,
 77,
 92,
 64,
 60,
 96,
 71,
 69,
 40,
 57,
 52,
 56,
 49,
 60,
 35,
 87,
 93,
 57,
 51,
 112,
 37,
 80,
 48,
 72,
 47,
 57,
 56,
 68,
 48,
 64,
 70,
 65,
 41,
 60,
 33,
 56,
 47,
 84,
 79,
 83,
 46,
 68,
 56,
 86,
 54,
 70,
 23,
 49,
 88,
 93,
 45,
 55,
 54,
 62,
 109,
 79,
 61,
 58,
 62,
 77,
 56,
 34,
 64,
 52,
 38,
 58,
 57,
 77,
 35,
 71,
 81,
 36,
 73,
 57,
 69,
 98,
 44,
 61,
 80,
 38,
 46,
 64,
 28,
 64,
 56,
 61,
 66,
 70,
 54,
 28,
 59,
 62,
 60,
 55,
 39,
 59,
 40,
 83,
 64,
 44,
 74,
 56,
 87,
 67,
 38,
 59,
 90,
 17,
 72,
 53,
 34,
 87,
 36,
 54,
 66,
 57,
 83,
 57,
 67,
 59,
 51,
 55,
 49,
 50,
 43,
 53,
 58,
 25,
 61,
 88,
 62,
 52,
 80,
 63,


In [34]:
len(phrase_len)

1000

(3) 데이터 프레임 생성

In [35]:
df_phrase_len = pd.DataFrame(phrase_len)

In [36]:
df_phrase_len

,0
0,105
1,40
2,40
3,51
4,16
...,...
995,67
996,120
997,70
998,99


(4) 데이터 프레임 합치기

In [37]:
df = pd.concat([df_phrase_len, df], axis = 1)

In [38]:
df

,0,sentence,Unnamed: 0,id,nationality,sex,title,content,score
0,105,18,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80
1,40,10,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70
2,40,7,3,A100000_v03.txt_1.txt,중국,남자,여행 계획,저는 여행을 제주도에서 갈 싶습니다. 저는 친구 같이 갈 겁니다. 6월 15일 갈 ...,50
3,51,9,4,A100001_v01.txt_1.txt,중국,남자,주말 이야기,주말에 도서관에서 공부했어요. 친구하고 노래방에서 노래했어요. 친구하고 식당에서 비...,60
4,16,4,5,A100001_v02.txt_1.txt,중국,남자,주말 이야기,어제 저는 학교에 갔습니다. 학교에서 친구하고를 다닙니다. 학교에 학생 식당 있습니다.,20
...,...,...,...,...,...,...,...,...,...
995,67,7,1007,A200079_v01.txt_1.txt,일본,여자,사진기 빌리기,하지만 빌리 씨는 사진기를 없었습니다. 그래서 빌리 씨는 아노코 씨에게 이야기하고 ...,90
996,120,15,1008,A200079_v01.txt_2.txt,일본,여자,우리 나라의 계절과 날씨,내 나라는 일본입니다. 일본의 계절은 뚜렷합니다. 봄은 다뜻합니다. 꽃을 많이 피나...,80
997,70,11,1009,A200079_v02.txt_1.txt,스웨덴,남자,편지,잘 지냈어요? 저도 잘 지냈어요. 한국 생활은 아주 재미있고 편해요. 한국어 공부를...,80
998,99,14,1010,A200079_v02.txt_2.txt,스웨덴,남자,기억에 남는 여행,지난 주말에 친구들하고 이탈리아에서 축구 경기를 보러 갔습니다. 우리는 영국에서 비...,90


(5) 데이터 프레임 열 이름 설정

In [39]:
df = df.rename(columns={0:'phrase'})

In [40]:
df

,phrase,sentence,Unnamed: 0,id,nationality,sex,title,content,score
0,105,18,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80
1,40,10,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70
2,40,7,3,A100000_v03.txt_1.txt,중국,남자,여행 계획,저는 여행을 제주도에서 갈 싶습니다. 저는 친구 같이 갈 겁니다. 6월 15일 갈 ...,50
3,51,9,4,A100001_v01.txt_1.txt,중국,남자,주말 이야기,주말에 도서관에서 공부했어요. 친구하고 노래방에서 노래했어요. 친구하고 식당에서 비...,60
4,16,4,5,A100001_v02.txt_1.txt,중국,남자,주말 이야기,어제 저는 학교에 갔습니다. 학교에서 친구하고를 다닙니다. 학교에 학생 식당 있습니다.,20
...,...,...,...,...,...,...,...,...,...
995,67,7,1007,A200079_v01.txt_1.txt,일본,여자,사진기 빌리기,하지만 빌리 씨는 사진기를 없었습니다. 그래서 빌리 씨는 아노코 씨에게 이야기하고 ...,90
996,120,15,1008,A200079_v01.txt_2.txt,일본,여자,우리 나라의 계절과 날씨,내 나라는 일본입니다. 일본의 계절은 뚜렷합니다. 봄은 다뜻합니다. 꽃을 많이 피나...,80
997,70,11,1009,A200079_v02.txt_1.txt,스웨덴,남자,편지,잘 지냈어요? 저도 잘 지냈어요. 한국 생활은 아주 재미있고 편해요. 한국어 공부를...,80
998,99,14,1010,A200079_v02.txt_2.txt,스웨덴,남자,기억에 남는 여행,지난 주말에 친구들하고 이탈리아에서 축구 경기를 보러 갔습니다. 우리는 영국에서 비...,90


##### 6) train 데이터 불러오기

In [42]:
df[["phrase", "sentence"]]

,phrase,sentence
0,105,18
1,40,10
2,40,7
3,51,9
4,16,4
...,...,...
995,67,7
996,120,15
997,70,11
998,99,14


In [43]:
df[["phrase", "sentence"]].values

array([[105,  18],
       [ 40,  10],
       [ 40,   7],
       ...,
       [ 70,  11],
       [ 99,  14],
       [ 81,   9]], dtype=int64)

In [44]:
df[["phrase", "sentence"]].iloc[0]

phrase      105
sentence     18
Name: 0, dtype: int64

In [45]:
X_train = torch.tensor(df[["phrase", "sentence"]].values).reshape(-1, 2).to(torch.float32)
y_train = torch.tensor(df["score"].values).reshape(-1, 1).to(torch.float32)

In [46]:
X_train

tensor([[105.,  18.],
        [ 40.,  10.],
        [ 40.,   7.],
        ...,
        [ 70.,  11.],
        [ 99.,  14.],
        [ 81.,   9.]])

In [47]:
y_train

tensor([[ 80.],
        [ 70.],
        [ 50.],
        [ 60.],
        [ 20.],
        [ 70.],
        [ 50.],
        [ 80.],
        [ 90.],
        [ 80.],
        [ 80.],
        [ 20.],
        [ 70.],
        [ 90.],
        [ 80.],
        [ 90.],
        [ 70.],
        [ 90.],
        [ 80.],
        [ 60.],
        [ 70.],
        [100.],
        [ 70.],
        [ 90.],
        [ 80.],
        [ 70.],
        [ 90.],
        [ 90.],
        [ 60.],
        [ 90.],
        [ 80.],
        [ 60.],
        [ 80.],
        [ 80.],
        [ 60.],
        [ 80.],
        [ 50.],
        [ 70.],
        [ 70.],
        [ 70.],
        [ 70.],
        [ 90.],
        [ 70.],
        [ 70.],
        [ 80.],
        [ 90.],
        [ 90.],
        [ 80.],
        [ 90.],
        [ 80.],
        [100.],
        [ 90.],
        [ 80.],
        [ 90.],
        [ 70.],
        [ 70.],
        [ 70.],
        [ 90.],
        [ 80.],
        [ 70.],
        [ 80.],
        [ 80.],
        

##### 7) 모델 정의 (MultivariableLinearRegressionModel)

In [41]:
class MultivariableLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
    def forward(self, x):
        return self.linear(x)
    
model = MultivariableLinearRegressionModel()

##### 8) Optimizer 정의

In [48]:
optimizer = optim.SGD(model.parameters(), lr=0.000012, momentum=0.9)

##### 9) 모델 학습

In [49]:
# (1) Epoch 순회
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # 1] 모델 예측
    y_pred = model(X_train)
    # 2] Cost 계산
    cost = F.mse_loss(y_pred, y_train)
    
    # 3] 모델 역전파
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 4] 결과 출력
    print('Epoch {:4d}/{} Cost:{:.6f}'.format(epoch, nb_epochs, cost.item())) #  hypothesis: {} , pred.squeeze().detach()

Epoch    0/20 Cost:10797.070312
Epoch    1/20 Cost:8216.898438
Epoch    2/20 Cost:4618.129883
Epoch    3/20 Cost:1812.099976
Epoch    4/20 Cost:882.546753
Epoch    5/20 Cost:1766.735229
Epoch    6/20 Cost:3503.386719
Epoch    7/20 Cost:4893.761719
Epoch    8/20 Cost:5158.899414
Epoch    9/20 Cost:4257.220703
Epoch   10/20 Cost:2764.385986
Epoch   11/20 Cost:1460.088135
Epoch   12/20 Cost:889.007935
Epoch   13/20 Cost:1125.621704
Epoch   14/20 Cost:1824.539795
Epoch   15/20 Cost:2476.337402
Epoch   16/20 Cost:2698.846436
Epoch   17/20 Cost:2408.011719
Epoch   18/20 Cost:1803.717651
Epoch   19/20 Cost:1212.983398
Epoch   20/20 Cost:897.911438


# 3. DataLoader 구현

##### 7) DataLoader

(1) CustomDataset

In [50]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x, y

dataset = CustomDataset(X_train, y_train)

(2) DataLoader

In [51]:
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True)

##### 8) 모델 정의 (MultivariableLinearRegressionModel)

In [ ]:
class MultivariableLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
    def forward(self, x):
        return self.linear(x)
    
model = MultivariableLinearRegressionModel()

##### 9) Optimizer 정의

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.000012, momentum=0.9)

##### 10) 모델 학습

In [52]:
# (1) Epoch 순회
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        # 1] 모델 예측
        y_pred = model(X_train)
        # 2] Cost 계산
        cost = F.mse_loss(y_pred, y_train)

        # 3] 모델 역전파
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 4] 결과 출력
        print('Epoch {:4d}/{} Batch {}/{} Cost:{:.6f}'.format(epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item())) #  hypothesis: {} , pred.squeeze().detach()

Epoch    0/20 Batch 1/250 Cost:933.408142
Epoch    0/20 Batch 2/250 Cost:1204.024292
Epoch    0/20 Batch 3/250 Cost:1498.897217
Epoch    0/20 Batch 4/250 Cost:1637.536987
Epoch    0/20 Batch 5/250 Cost:1556.195068
Epoch    0/20 Batch 6/250 Cost:1318.078247
Epoch    0/20 Batch 7/250 Cost:1056.641846
Epoch    0/20 Batch 8/250 Cost:893.715881
Epoch    0/20 Batch 9/250 Cost:879.157043
Epoch    0/20 Batch 10/250 Cost:978.835388
Epoch    0/20 Batch 11/250 Cost:1107.921997
Epoch    0/20 Batch 12/250 Cost:1183.823242
Epoch    0/20 Batch 13/250 Cost:1167.927856
Epoch    0/20 Batch 14/250 Cost:1076.912476
Epoch    0/20 Batch 15/250 Cost:963.604492
Epoch    0/20 Batch 16/250 Cost:882.978943
Epoch    0/20 Batch 17/250 Cost:863.526672
Epoch    0/20 Batch 18/250 Cost:897.629639
Epoch    0/20 Batch 19/250 Cost:952.258972
Epoch    0/20 Batch 20/250 Cost:990.658630
Epoch    0/20 Batch 21/250 Cost:991.821472
Epoch    0/20 Batch 22/250 Cost:958.207581
Epoch    0/20 Batch 23/250 Cost:909.984741
Epoch    0

Epoch    1/20 Batch 135/250 Cost:708.592712
Epoch    1/20 Batch 136/250 Cost:708.347351
Epoch    1/20 Batch 137/250 Cost:708.102783
Epoch    1/20 Batch 138/250 Cost:707.858887
Epoch    1/20 Batch 139/250 Cost:707.615540
Epoch    1/20 Batch 140/250 Cost:707.372864
Epoch    1/20 Batch 141/250 Cost:707.130859
Epoch    1/20 Batch 142/250 Cost:706.889526
Epoch    1/20 Batch 143/250 Cost:706.648682
Epoch    1/20 Batch 144/250 Cost:706.408569
Epoch    1/20 Batch 145/250 Cost:706.169006
Epoch    1/20 Batch 146/250 Cost:705.930176
Epoch    1/20 Batch 147/250 Cost:705.691895
Epoch    1/20 Batch 148/250 Cost:705.454224
Epoch    1/20 Batch 149/250 Cost:705.217224
Epoch    1/20 Batch 150/250 Cost:704.980957
Epoch    1/20 Batch 151/250 Cost:704.745239
Epoch    1/20 Batch 152/250 Cost:704.510071
Epoch    1/20 Batch 153/250 Cost:704.275513
Epoch    1/20 Batch 154/250 Cost:704.041626
Epoch    1/20 Batch 155/250 Cost:703.808411
Epoch    1/20 Batch 156/250 Cost:703.575745
Epoch    1/20 Batch 157/250 Cost

Epoch    3/20 Batch 164/250 Cost:637.486023
Epoch    3/20 Batch 165/250 Cost:637.420471
Epoch    3/20 Batch 166/250 Cost:637.355042
Epoch    3/20 Batch 167/250 Cost:637.289734
Epoch    3/20 Batch 168/250 Cost:637.224670
Epoch    3/20 Batch 169/250 Cost:637.159668
Epoch    3/20 Batch 170/250 Cost:637.094849
Epoch    3/20 Batch 171/250 Cost:637.030273
Epoch    3/20 Batch 172/250 Cost:636.965759
Epoch    3/20 Batch 173/250 Cost:636.901367
Epoch    3/20 Batch 174/250 Cost:636.837097
Epoch    3/20 Batch 175/250 Cost:636.773071
Epoch    3/20 Batch 176/250 Cost:636.709045
Epoch    3/20 Batch 177/250 Cost:636.645325
Epoch    3/20 Batch 178/250 Cost:636.581604
Epoch    3/20 Batch 179/250 Cost:636.518188
Epoch    3/20 Batch 180/250 Cost:636.454834
Epoch    3/20 Batch 181/250 Cost:636.391541
Epoch    3/20 Batch 182/250 Cost:636.328552
Epoch    3/20 Batch 183/250 Cost:636.265625
Epoch    3/20 Batch 184/250 Cost:636.202881
Epoch    3/20 Batch 185/250 Cost:636.140198
Epoch    3/20 Batch 186/250 Cost

Epoch    5/20 Batch 191/250 Cost:616.458252
Epoch    5/20 Batch 192/250 Cost:616.434387
Epoch    5/20 Batch 193/250 Cost:616.410522
Epoch    5/20 Batch 194/250 Cost:616.386597
Epoch    5/20 Batch 195/250 Cost:616.362793
Epoch    5/20 Batch 196/250 Cost:616.339111
Epoch    5/20 Batch 197/250 Cost:616.315491
Epoch    5/20 Batch 198/250 Cost:616.291748
Epoch    5/20 Batch 199/250 Cost:616.268066
Epoch    5/20 Batch 200/250 Cost:616.244507
Epoch    5/20 Batch 201/250 Cost:616.220886
Epoch    5/20 Batch 202/250 Cost:616.197388
Epoch    5/20 Batch 203/250 Cost:616.173767
Epoch    5/20 Batch 204/250 Cost:616.150391
Epoch    5/20 Batch 205/250 Cost:616.126953
Epoch    5/20 Batch 206/250 Cost:616.103516
Epoch    5/20 Batch 207/250 Cost:616.080139
Epoch    5/20 Batch 208/250 Cost:616.056763
Epoch    5/20 Batch 209/250 Cost:616.033508
Epoch    5/20 Batch 210/250 Cost:616.010193
Epoch    5/20 Batch 211/250 Cost:615.986938
Epoch    5/20 Batch 212/250 Cost:615.963684
Epoch    5/20 Batch 213/250 Cost

Epoch    7/20 Batch 226/250 Cost:606.946106
Epoch    7/20 Batch 227/250 Cost:606.932129
Epoch    7/20 Batch 228/250 Cost:606.918213
Epoch    7/20 Batch 229/250 Cost:606.904297
Epoch    7/20 Batch 230/250 Cost:606.890381
Epoch    7/20 Batch 231/250 Cost:606.876404
Epoch    7/20 Batch 232/250 Cost:606.862488
Epoch    7/20 Batch 233/250 Cost:606.848633
Epoch    7/20 Batch 234/250 Cost:606.834778
Epoch    7/20 Batch 235/250 Cost:606.820801
Epoch    7/20 Batch 236/250 Cost:606.806946
Epoch    7/20 Batch 237/250 Cost:606.793091
Epoch    7/20 Batch 238/250 Cost:606.779236
Epoch    7/20 Batch 239/250 Cost:606.765320
Epoch    7/20 Batch 240/250 Cost:606.751526
Epoch    7/20 Batch 241/250 Cost:606.737610
Epoch    7/20 Batch 242/250 Cost:606.723877
Epoch    7/20 Batch 243/250 Cost:606.710022
Epoch    7/20 Batch 244/250 Cost:606.696106
Epoch    7/20 Batch 245/250 Cost:606.682373
Epoch    7/20 Batch 246/250 Cost:606.668579
Epoch    7/20 Batch 247/250 Cost:606.654724
Epoch    7/20 Batch 248/250 Cost

...

Epoch   12/20 Batch 19/250 Cost:594.642151
Epoch   12/20 Batch 20/250 Cost:594.631409
Epoch   12/20 Batch 21/250 Cost:594.620605
Epoch   12/20 Batch 22/250 Cost:594.609863
Epoch   12/20 Batch 23/250 Cost:594.599060
Epoch   12/20 Batch 24/250 Cost:594.588318
Epoch   12/20 Batch 25/250 Cost:594.577576
Epoch   12/20 Batch 26/250 Cost:594.566772
Epoch   12/20 Batch 27/250 Cost:594.556030
Epoch   12/20 Batch 28/250 Cost:594.545166
Epoch   12/20 Batch 29/250 Cost:594.534363
Epoch   12/20 Batch 30/250 Cost:594.523621
Epoch   12/20 Batch 31/250 Cost:594.512817
Epoch   12/20 Batch 32/250 Cost:594.502075
Epoch   12/20 Batch 33/250 Cost:594.491333
Epoch   12/20 Batch 34/250 Cost:594.480591
Epoch   12/20 Batch 35/250 Cost:594.469727
Epoch   12/20 Batch 36/250 Cost:594.458923
Epoch   12/20 Batch 37/250 Cost:594.448242
Epoch   12/20 Batch 38/250 Cost:594.437500
Epoch   12/20 Batch 39/250 Cost:594.426758
Epoch   12/20 Batch 40/250 Cost:594.415955
Epoch   12/20 Batch 41/250 Cost:594.405212
Epoch   12/

Epoch   14/20 Batch 50/250 Cost:589.018555
Epoch   14/20 Batch 51/250 Cost:589.008057
Epoch   14/20 Batch 52/250 Cost:588.997742
Epoch   14/20 Batch 53/250 Cost:588.987305
Epoch   14/20 Batch 54/250 Cost:588.976868
Epoch   14/20 Batch 55/250 Cost:588.966431
Epoch   14/20 Batch 56/250 Cost:588.955994
Epoch   14/20 Batch 57/250 Cost:588.945557
Epoch   14/20 Batch 58/250 Cost:588.935120
Epoch   14/20 Batch 59/250 Cost:588.924683
Epoch   14/20 Batch 60/250 Cost:588.914246
Epoch   14/20 Batch 61/250 Cost:588.903870
Epoch   14/20 Batch 62/250 Cost:588.893494
Epoch   14/20 Batch 63/250 Cost:588.883057
Epoch   14/20 Batch 64/250 Cost:588.872620
Epoch   14/20 Batch 65/250 Cost:588.862183
Epoch   14/20 Batch 66/250 Cost:588.851807
Epoch   14/20 Batch 67/250 Cost:588.841309
Epoch   14/20 Batch 68/250 Cost:588.830933
Epoch   14/20 Batch 69/250 Cost:588.820557
Epoch   14/20 Batch 70/250 Cost:588.810120
Epoch   14/20 Batch 71/250 Cost:588.799683
Epoch   14/20 Batch 72/250 Cost:588.789246
Epoch   14/

Epoch   16/20 Batch 79/250 Cost:583.572998
Epoch   16/20 Batch 80/250 Cost:583.562866
Epoch   16/20 Batch 81/250 Cost:583.552612
Epoch   16/20 Batch 82/250 Cost:583.542419
Epoch   16/20 Batch 83/250 Cost:583.532349
Epoch   16/20 Batch 84/250 Cost:583.522217
Epoch   16/20 Batch 85/250 Cost:583.512024
Epoch   16/20 Batch 86/250 Cost:583.501892
Epoch   16/20 Batch 87/250 Cost:583.491638
Epoch   16/20 Batch 88/250 Cost:583.481506
Epoch   16/20 Batch 89/250 Cost:583.471375
Epoch   16/20 Batch 90/250 Cost:583.461182
Epoch   16/20 Batch 91/250 Cost:583.450989
Epoch   16/20 Batch 92/250 Cost:583.440857
Epoch   16/20 Batch 93/250 Cost:583.430664
Epoch   16/20 Batch 94/250 Cost:583.420532
Epoch   16/20 Batch 95/250 Cost:583.410400
Epoch   16/20 Batch 96/250 Cost:583.400208
Epoch   16/20 Batch 97/250 Cost:583.390076
Epoch   16/20 Batch 98/250 Cost:583.379822
Epoch   16/20 Batch 99/250 Cost:583.369751
Epoch   16/20 Batch 100/250 Cost:583.359619
Epoch   16/20 Batch 101/250 Cost:583.349365
Epoch   1

Epoch   18/20 Batch 118/250 Cost:578.156006
Epoch   18/20 Batch 119/250 Cost:578.146118
Epoch   18/20 Batch 120/250 Cost:578.136169
Epoch   18/20 Batch 121/250 Cost:578.126221
Epoch   18/20 Batch 122/250 Cost:578.116333
Epoch   18/20 Batch 123/250 Cost:578.106384
Epoch   18/20 Batch 124/250 Cost:578.096436
Epoch   18/20 Batch 125/250 Cost:578.086487
Epoch   18/20 Batch 126/250 Cost:578.076538
Epoch   18/20 Batch 127/250 Cost:578.066711
Epoch   18/20 Batch 128/250 Cost:578.056763
Epoch   18/20 Batch 129/250 Cost:578.046814
Epoch   18/20 Batch 130/250 Cost:578.036865
Epoch   18/20 Batch 131/250 Cost:578.026917
Epoch   18/20 Batch 132/250 Cost:578.017029
Epoch   18/20 Batch 133/250 Cost:578.007141
Epoch   18/20 Batch 134/250 Cost:577.997192
Epoch   18/20 Batch 135/250 Cost:577.987244
Epoch   18/20 Batch 136/250 Cost:577.977234
Epoch   18/20 Batch 137/250 Cost:577.967346
Epoch   18/20 Batch 138/250 Cost:577.957458
Epoch   18/20 Batch 139/250 Cost:577.947571
Epoch   18/20 Batch 140/250 Cost

Epoch   20/20 Batch 173/250 Cost:572.707520
Epoch   20/20 Batch 174/250 Cost:572.697754
Epoch   20/20 Batch 175/250 Cost:572.688110
Epoch   20/20 Batch 176/250 Cost:572.678467
Epoch   20/20 Batch 177/250 Cost:572.668762
Epoch   20/20 Batch 178/250 Cost:572.658997
Epoch   20/20 Batch 179/250 Cost:572.649292
Epoch   20/20 Batch 180/250 Cost:572.639648
Epoch   20/20 Batch 181/250 Cost:572.629883
Epoch   20/20 Batch 182/250 Cost:572.620178
Epoch   20/20 Batch 183/250 Cost:572.610474
Epoch   20/20 Batch 184/250 Cost:572.600830
Epoch   20/20 Batch 185/250 Cost:572.591125
Epoch   20/20 Batch 186/250 Cost:572.581360
Epoch   20/20 Batch 187/250 Cost:572.571716
Epoch   20/20 Batch 188/250 Cost:572.562012
Epoch   20/20 Batch 189/250 Cost:572.552368
Epoch   20/20 Batch 190/250 Cost:572.542603
Epoch   20/20 Batch 191/250 Cost:572.532959
Epoch   20/20 Batch 192/250 Cost:572.523254
Epoch   20/20 Batch 193/250 Cost:572.513550
Epoch   20/20 Batch 194/250 Cost:572.503845
Epoch   20/20 Batch 195/250 Cost

# 4. Automatic Writing Evalutation System

##### 7) DataLoader

(1) CustomDataset

In [53]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = torch.FloatTensor(self.y_data[idx])
        
        return x, y

dataset = CustomDataset(df['content'], y_train)

##### (2) DataLoader

In [54]:
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True)

In [55]:
dataloader_iter = iter(dataloader)
batch_idx, samples = next(dataloader_iter)

In [56]:
batch_idx, samples

(('그래서 동대문 근처 식당에서 생일 파티를 합니다. 아버지하고 어머니하고 선생님하고 우리 반 친구가 옵니다. 아버지가 선물을 줍니다. 한국어 사전하고 컴퓨터를 줍니다. 한국어 사전이 아주 좋아요. 그래서 저는 한국어를 열심히 공부합니다. 어머니도 선물을 줍니다. 신발하고 치마를 줍니다. 그 치마가 너무 예쁩니다. 선생님하고 우리 반 친구는 생일 축하 노래를 부릅니다. 노래를 잘합니다. 우리는 사진을 많이 찍습니다. 그리고 음식을 많이 먹습니다. 불고기하고 피자를 먹습니다. 료코 씨는 케이크를 삽니다. 그 케이크가 맛있습니다. 저는 아주 행복합니다! 생일 파티를 좋아합니다.',
  '그리고 생일은 죽하하고 약속 했습니다. 나중에 빌리 씨와 나오코 씨가 같이 극장에 갔습니다. 재미있는 영화 보고 식당에 갔습니다. 식당에서 많이 밥을 먹었습니다. 너무 맛있었습니다. 후에 빌리 씨와 나오코 씨가 친구들하고 만나서 같이 술집에 갔습니다. 술집에서 술을 마시고 집에 갔습니다. 집에서 인터넷을 하고 잤습니다.',
  '나오코 씨는 아직 버스를 타고 있었습니다. 그리고 10분 후에 나오코 씨가 왔습니다. 왕영 씨에게 아침 늦게 일어나서 늦었습니다. "미안해요"라고 말했습니다. 두 사람은 같이 나오코 씨 집에 갔습니다. 오늘은 함께 요리를 하는 약속을 하기 때문에 집에 도착 하기 전에 야채를 사고 갔습니다. 두 사람은 요리를 잘 할 수 있습니다. 맛있는 음식을 만들고 재미있는 시간을 보냈습니다. 식사 끝난 후에 왕영 씨가 나오코 씨에게서 선물을 받았습니다. 다음 주은 왕영 씨 생일입니다. 왕영 씨는 깜짝 놀랐습니다. 그리고 아주 기뻐했습니다.',
  '저는 춤을 추는 젓을 좋아합니다. 어린 날 부터 저는 음악을 너무 좋아서 음악 상관의 취미 다 좋아합니다. 저는 열한 살 부터 학교에서 친구하고 춤을 배웠습니다. 항상 수업 끝난 후에 시간이 있으면 친구 집에 가서 춤을 연습 했습니다. 춤을 출 때 음악을 항상 큰 소리를 켰습니다. 음악을 큰소리 들을 때 기분이 제일 좋기 때

##### 8) 모델 정의 (MultivariableLinearRegressionModel)

In [ ]:
class MultiVariableLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        
    def forward(self, x):
        return self.linear(x)

##### 9) Optimizer 정의

In [ ]:
model = MultiVariableLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=0.00005, momentum=0.9)

##### 10) 모델 학습

In [ ]:
# (1) Epoch 순회
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        # 1] 모델 예측
        y_pred = model(X_train)
        # 2] Cost 계산
        cost = F.mse_loss(y_pred, y_train)

        # 3] 모델 역전파
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 4] 결과 출력
        print('Epoch {:4d}/{} Batch {}/{} Cost:{:.6f}'.format(epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item())) #  hypothesis: {} , pred.squeeze().detach()